## 1. B树和B+树的区别  

都是一种多路平衡树，和平衡二叉树类似，但是B/B+树更加矮胖，减少了树的高度；  
B+树的内部节点是作为索引，不存储数据的，所有的数据都是存储在叶子节点中；而B树是中间节点也存储数据；  

#### B+ 树的三个优点：
1. 层级更低，IO 次数更少
2. 每次都需要查询到叶子节点，查询性能稳定; 而B树，每个节点都可以查找到数据，所以不稳定。
3. 叶子节点形成有序链表，范围查询方便

## 2. 分布式Session

使用分布式session会遇到 **session不一致的问题**（就是说在一台服务器上登录，下一次访问时，请求被分配到另外一个服务器，需要重新登录获取session这种情况

### 解决session不一致问题 （4种）

**1. Session 复制**（不推荐使用）  
将一个服务器的session复制到其他服务器中  
**缺点：**
1. Session 复制传输需要占用内网带宽，如果机器很多，可能会形成网络风暴，复制性能也会呈指数级下降
2. 将Session存放中Tomcat中，容易受到Tomcat内存的限制，不好水平拓展

**2. Session 前端存储/客户端存储**  
直接将信息存储在Cookie中，每次请求过来的时候，判断Cookie的用户信息  
**缺点：**  
1. 数据存储在端上，并在网络传输，存在泄漏、篡改、窃取等安全隐患
2. 每次http请求都携带session，占外网带宽
3. Cookie数量和长度有限制

**3. Session粘滞**  
将用户的每次请求都**通过某种方法**（Hash策略）强制分发到某**一个**Web服务器上，只要这个Web服务器上存储了对应Session数据，就可以实现会话跟踪。  

**优点：**

* 只需要改nginx配置，不需要修改应用代码
* 负载均衡，只要hash属性是均匀的，多台服务端的负载是均衡的
* 可以支持服务端水平扩展（session同步法是不行的，受内存限制）

**缺点：**

* 如果服务端重启，一部分session会丢失，产生业务影响，例如部分用户重新登录
* 如果服务端水平扩展，rehash后session重新分布，也会有一部分用户路由不到正确的session


**4. 后端集中存储**  
在**单独**的服务器或服务器集群上使用缓存技术（如Redis存储Session数据），集中管理所有的Session，所有的Web服务器都从这个存储介质中存取对应的Session，实现Session共享  

项目中使用的就是 使用Redis集中存储

**优点：**
* 没有安全隐患
* 可以水平扩展，数据库/缓存水平切分即可
* 服务端重启或者扩容都不会有session丢失  

**不足：**  

增加了一次网络调用，并且需要修改应用代码

![](img/session.png)